### Инициализация Keras

In [1]:
import os

os.environ["KERAS_BACKEND"] = "jax"
import keras

print(keras.__version__)

3.9.2


#### Загрузка набора данных для задачи классификации

В данном примере используется фрагмент набора  данных Cats and Dogs Classification Dataset

В наборе данных два класса: кошки и собаки

Ссылка: https://www.kaggle.com/datasets/bhavikjikadara/dog-and-cat-classification-dataset

In [10]:
import kagglehub

path = kagglehub.dataset_download("bhavikjikadara/dog-and-cat-classification-dataset")
path

Resuming download from 312475648 bytes (500272489 bytes left)...
Resuming download from https://www.kaggle.com/api/v1/datasets/download/bhavikjikadara/dog-and-cat-classification-dataset?dataset_version_number=1 (312475648/812748137) bytes left.


100%|██████████| 775M/775M [01:03<00:00, 7.86MB/s]

Extracting files...


'/Users/user/.cache/kagglehub/datasets/bhavikjikadara/dog-and-cat-classification-dataset/versions/1'

#### Формирование выборок

In [11]:
from keras.src.legacy.preprocessing.image import ImageDataGenerator

batch_size = 64

data_loader = ImageDataGenerator(validation_split=0.2)

train = data_loader.flow_from_directory(
    directory=path,
    target_size=(224, 224),
    color_mode="rgb",
    class_mode="categorical",
    batch_size=batch_size,
    shuffle=True,
    seed=9,
    subset="training",
)

valid = data_loader.flow_from_directory(
    directory=path,
    target_size=(224, 224),
    color_mode="rgb",
    class_mode="categorical",
    batch_size=batch_size,
    shuffle=True,
    seed=9,
    subset="validation",
)

Found 19999 images belonging to 1 classes.
Found 4999 images belonging to 1 classes.


### Архитектура AlexNet

#### Проектирование архитектуры AlexNet

In [3]:
from keras.api.models import Sequential
from keras.api.layers import InputLayer, Conv2D, MaxPooling2D, Dropout, Flatten, Dense, BatchNormalization

alexnet_model = Sequential()

# Входной слой
alexnet_model.add(InputLayer(shape=(224, 224, 3)))

# Первый скрытый слой
alexnet_model.add(Conv2D(96, kernel_size=(11, 11), strides=(4, 4), activation="relu"))
alexnet_model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
alexnet_model.add(BatchNormalization())

# Второй скрытый слой
alexnet_model.add(Conv2D(256, kernel_size=(5, 5), activation="relu"))
alexnet_model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
alexnet_model.add(BatchNormalization())

# Третий скрытый слой
alexnet_model.add(Conv2D(256, kernel_size=(3, 3), activation="relu"))

# Четвертый скрытый слой
alexnet_model.add(Conv2D(384, kernel_size=(3, 3), activation="relu"))

# Пятый скрытый слой
alexnet_model.add(Conv2D(384, kernel_size=(3, 3), activation="relu"))
alexnet_model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
alexnet_model.add(BatchNormalization())

# Шестой скрытый слой
alexnet_model.add(Flatten())
alexnet_model.add(Dense(4096, activation="tanh"))
alexnet_model.add(Dropout(0.5))

# Седьмой скрытый слой
alexnet_model.add(Dense(4096, activation="tanh"))
alexnet_model.add(Dropout(0.5))

# Выходной слой
alexnet_model.add(Dense(len(train.class_indices), activation="softmax"))

alexnet_model.summary()

W0000 00:00:1745616435.424692 2474885 mps_client.cc:510] WARNING: JAX Apple GPU support is experimental and not all JAX functionality is correctly supported!
I0000 00:00:1745616435.449215 2474885 service.cc:145] XLA service 0x15fe0b760 initialized for platform METAL (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1745616435.449229 2474885 service.cc:153]   StreamExecutor device (0): Metal, <undefined>
I0000 00:00:1745616435.451710 2474885 mps_client.cc:406] Using Simple allocator.
I0000 00:00:1745616435.451728 2474885 mps_client.cc:384] XLA backend will use up to 28990554112 bytes on device 0 for SimpleAllocator.


Metal device set to: Apple M3 Pro

systemMemory: 36.00 GB
maxCacheSize: 13.50 GB



Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 54, 54, 96)     │        34,944 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 26, 26, 96)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 26, 26, 96)     │           384 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 22, 22, 256)    │       614,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 10, 10, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 10, 10, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 8, 8, 256)      │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 6, 6, 384)      │       885,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 4, 4, 384)      │     1,327,488 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 1, 1, 384)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 1, 1, 384)      │         1,536 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 384)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 4096)           │     1,576,960 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 4096)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 4096)           │    16,781,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 4096)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 2)              │         8,194 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 21,821,698 (83.24 MB)

 Trainable params: 21,820,226 (83.24 MB)

 Non-trainable params: 1,472 (5.75 KB)

#### Обучение глубокой модели

In [4]:
alexnet_model.compile(
    loss="categorical_crossentropy",
    optimizer="adam",
    metrics=["accuracy"],
)

alexnet_model.fit(
    x=train,
    validation_data=valid,
    epochs=100
)

/Users/user/Projects/python/mai/.venv/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/100


/Users/user/Projects/python/mai/.venv/lib/python3.12/site-packages/jax/_src/interpreters/mlir.py:1153: UserWarning: Some donated buffers were not usable: ShapedArray(float32[11,11,3,96]), ShapedArray(float32[96]), ShapedArray(float32[96]), ShapedArray(float32[96]), ShapedArray(float32[5,5,96,256]), ShapedArray(float32[256]), ShapedArray(float32[256]), ShapedArray(float32[256]), ShapedArray(float32[3,3,256,256]), ShapedArray(float32[256]), ShapedArray(float32[3,3,256,384]), ShapedArray(float32[384]), ShapedArray(float32[3,3,384,384]), ShapedArray(float32[384]), ShapedArray(float32[384]), ShapedArray(float32[384]), ShapedArray(float32[384,4096]), ShapedArray(float32[4096]), ShapedArray(float32[4096,4096]), ShapedArray(float32[4096]), ShapedArray(float32[4096,2]), ShapedArray(float32[2]), ShapedArray(float32[96]), ShapedArray(float32[96]), ShapedArray(float32[256]), ShapedArray(float32[256]), ShapedArray(float32[384]), ShapedArray(float32[384]), ShapedArray(uint32[2]), ShapedArray(uint32[

244/313 ━━━━━━━━━━━━━━━━━━━━ 7s 110ms/step - accuracy: 0.5025 - loss: 3.7129

/Users/user/Projects/python/mai/.venv/lib/python3.12/site-packages/PIL/TiffImagePlugin.py:900: UserWarning: Truncated File Read
  warnings.warn(str(msg))


312/313 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step - accuracy: 0.5027 - loss: 3.2935

/Users/user/Projects/python/mai/.venv/lib/python3.12/site-packages/jax/_src/interpreters/mlir.py:1153: UserWarning: Some donated buffers were not usable: ShapedArray(float32[11,11,3,96]), ShapedArray(float32[96]), ShapedArray(float32[96]), ShapedArray(float32[96]), ShapedArray(float32[5,5,96,256]), ShapedArray(float32[256]), ShapedArray(float32[256]), ShapedArray(float32[256]), ShapedArray(float32[3,3,256,256]), ShapedArray(float32[256]), ShapedArray(float32[3,3,256,384]), ShapedArray(float32[384]), ShapedArray(float32[3,3,384,384]), ShapedArray(float32[384]), ShapedArray(float32[384]), ShapedArray(float32[384]), ShapedArray(float32[384,4096]), ShapedArray(float32[4096]), ShapedArray(float32[4096,4096]), ShapedArray(float32[4096]), ShapedArray(float32[4096,2]), ShapedArray(float32[2]), ShapedArray(float32[96]), ShapedArray(float32[96]), ShapedArray(float32[256]), ShapedArray(float32[256]), ShapedArray(float32[384]), ShapedArray(float32[384]), ShapedArray(uint32[2]), ShapedArray(uint32[

313/313 ━━━━━━━━━━━━━━━━━━━━ 50s 129ms/step - accuracy: 0.5027 - loss: 3.2887 - val_accuracy: 0.4960 - val_loss: 0.7563
Epoch 2/100
313/313 ━━━━━━━━━━━━━━━━━━━━ 37s 112ms/step - accuracy: 0.5116 - loss: 0.8322 - val_accuracy: 0.5080 - val_loss: 0.9215
Epoch 3/100
313/313 ━━━━━━━━━━━━━━━━━━━━ 35s 112ms/step - accuracy: 0.5023 - loss: 0.8604 - val_accuracy: 0.4984 - val_loss: 0.7005
Epoch 4/100
313/313 ━━━━━━━━━━━━━━━━━━━━ 35s 112ms/step - accuracy: 0.5046 - loss: 0.8040 - val_accuracy: 0.5194 - val_loss: 0.8496
Epoch 5/100
313/313 ━━━━━━━━━━━━━━━━━━━━ 35s 113ms/step - accuracy: 0.5054 - loss: 0.8219 - val_accuracy: 0.5218 - val_loss: 0.7518
Epoch 6/100
313/313 ━━━━━━━━━━━━━━━━━━━━ 34s 109ms/step - accuracy: 0.5146 - loss: 0.8019 - val_accuracy: 0.4766 - val_loss: 0.8985
Epoch 7/100
313/313 ━━━━━━━━━━━━━━━━━━━━ 34s 109ms/step - accuracy: 0.5098 - loss: 0.8301 - val_accuracy: 0.4840 - val_loss: 0.7566
Epoch 8/100
313/313 ━━━━━━━━━━━━━━━━━━━━ 34s 110ms/step - accuracy: 0.5055 - loss: 0.819

#### Оценка качества модели

In [5]:
alexnet_model.evaluate(valid)

79/79 ━━━━━━━━━━━━━━━━━━━━ 4s 51ms/step - accuracy: 0.8671 - loss: 0.4777


[0.5021674036979675, 0.8677471280097961]